<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
BUCKET = '002_create_dataset'
PROJECT = '002_create_dataset'
REGION = 'us-central1'

In [2]:
import os

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [4]:
# Create SQL query using natality data after the year 2000

from google.cloud import bigquery

query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

In [5]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages

df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))

df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-7170969733900686954,331274
1,5896567601480310696,342825
2,-5742197815970064689,359684
3,-774501970389208065,349134
4,454960867574323744,346515


## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

In [6]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want

trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 5)) < 4 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 5)) = 4 AND RAND() < 0.0005"

traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()

print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 12799 examples in the train dataset and 4011 in the eval dataset


In [7]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,8.750147,True,21,1,30.0,3095933535584005890
1,6.311835,True,32,1,40.0,3095933535584005890
2,6.660165,False,31,1,38.0,3095933535584005890
3,7.561856,True,18,1,40.0,3095933535584005890
4,8.763375,True,41,1,38.0,3095933535584005890


In [8]:
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,12786.000000,12799.000000,12799.000000,12706.000000,1.279900e+04
mean,7.220086,27.392687,1.034143,38.600346,-2.462651e+16
std,1.296544,6.208886,0.188363,2.519074,5.057877e+18
min,0.562179,12.000000,1.000000,17.000000,-9.183606e+18
25%,6.503637,22.000000,1.000000,38.000000,-4.701948e+18
50%,7.312733,27.000000,1.000000,39.000000,-3.280124e+17
75%,8.024826,32.000000,1.000000,40.000000,3.545707e+18
max,11.624975,48.000000,4.000000,47.000000,8.535134e+18


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

In [9]:
import pandas as pd

def preprocess(df):
  # clean up data we don't want to train on

  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string

  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
    
  df['plurality'].replace(twins_etc, inplace = True)
  
  # create extra rows to simulate lack of ultrasound

  nous = df.copy(deep = True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [10]:
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.tail() # check the no ultrasound rows

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
12794,7.687519,Unknown,26,Single(1),46.0,74931465496927487
12795,8.280563,Unknown,24,Single(1),40.0,74931465496927487
12796,5.937049,Unknown,18,Single(1),36.0,74931465496927487
12797,9.812775,Unknown,24,Single(1),38.0,74931465496927487
12798,7.312733,Unknown,27,Single(1),39.0,74931465496927487


## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [11]:
traindf.to_csv('train.csv', index = False, header = False)
evaldf.to_csv('eval.csv', index = False, header = False)

In [12]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   7978 eval.csv
  25386 train.csv
  33364 total
==> eval.csv <==
7.932232186759999,False,32,Single(1),39.0,-7170969733900686954
8.000575487979999,False,26,Single(1),39.0,-5742197815970064689
7.25100379718,False,21,Single(1),41.0,-2700219941840496344
8.2452885988,False,30,Single(1),39.0,-2363238223526193234
5.37486994756,False,20,Single(1),45.0,-5742197815970064689
5.6879263596,False,19,Single(1),37.0,-9068386407968572094
7.3744626639,True,18,Single(1),44.0,2962886928991417684
2.31264912838,True,19,Single(1),30.0,7206972366848618664
8.12623897732,False,23,Single(1),43.0,-4979697502521811334
2.68743497378,True,30,Single(1),28.0,-1403073183891835564

==> train.csv <==
8.75014717878,True,21,Single(1),30.0,3095933535584005890
6.3118345610599995,True,32,Single(1),40.0,3095933535584005890
6.66016493502,False,31,Single(1),38.0,3095933535584005890
7.5618555866,True,18,Single(1),40.0,3095933535584005890
8.7633749145,True,41,Single(1),38.0,3095933535584005890
7.62578964258,False,27,Single(1),39.